In [1]:
import sys
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [2]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)


# Add 5 masses and 4 distance constraints to form a chain
N_MASSES = 10
LINK_LENGTH = 1.0

# Stilld fix the first mass at the origin
f1 = mss.add(Fix((0,0,0)))
previous_connector = f1

# Use for loop to add masses and constraints
# Such that we can change N_MASSES easily

for i in range(N_MASSES):
    initial_pos = (LINK_LENGTH * (i + 1), 0.0, 0.0)
    
    # add new mass
    current_mass_connector = mss.add(Mass(1, initial_pos)) 
    
    # add new constraint and connet it to the previous mass
    dc = DistanceConstraint(previous_connector, current_mass_connector, LINK_LENGTH)
    mss.addDistanceConstraint(dc)
    
    # refresh
    previous_connector = current_mass_connector




In [3]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []

for c in mss.constraints:
    pA = mss.fixes[c.c1.nr].pos if c.c1.type == 1 else mss.masses[c.c1.nr].pos
    pB = mss.fixes[c.c2.nr].pos if c.c2.type == 1 else mss.masses[c.c2.nr].pos

    springpos.append([pA, pB])


springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [4]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [ ]:
from time import sleep
for i in range(10000):
    mss.simulate (0.02, 1)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = tuple(m.pos)

    springpos = []
    for c in mss.constraints:
        if c.c1.type == 1:                      # FIX
            pA = mss.fixes[c.c1.nr].pos
        else:                                   # MASS
            pA = mss.masses[c.c1.nr].pos

        # get position of second constraint point
        if c.c2.type == 1:
            pB = mss.fixes[c.c2.nr].pos
        else:
            pB = mss.masses[c.c2.nr].pos
        springpos.append ([ pA, pB ])
    springs.geometry = LineSegmentsGeometry(positions=springpos)

    sleep(0.2
    
    )